In [8]:
import string
import re
import os

In [9]:
def clean_text(line):
    
    for char in string.punctuation:
        if char!='-':
            line = line.replace(char, ' ')
    line = re.sub(' +', ' ', line)
    line = line.strip()
    line = line.lower()
    for i in line:
        if i.isalpha():
            return line
    return ""  

In [10]:
def preprocess(annotation_file, txt_file, output_file):
    
    f = open(txt_file)
    text = ""
    for line in f.readlines():
        text = line.split()
    f.close()
    
    ann_file = open(annotation_file)
    num_start_process = {}
    num_mid_process = {}
    num_start_task = {}
    num_mid_task = {}
    num_start_material = {}
    num_mid_material = {}
    visited = {}
    for line in ann_file.readlines():
        split_line  = line.split()    
        if split_line[2].isnumeric():
            for i in range(int(split_line[2]),int(split_line[3]), 1):
                visited[int(split_line[2])] = 1

            if split_line[1] == 'Process':
                num_start_process[int(split_line[2])] = split_line[4]
                temp = []
                for i in range(5,len(split_line),1):
                    temp.append(split_line[i])
                if len(temp)>0:
                    num_mid_process[int(split_line[2])] = []
                    num_mid_process[int(split_line[2])].append(temp)

            elif split_line[1] == 'Material':
                num_start_material[int(split_line[2])] = split_line[4]
                temp = []
                for i in range(5,len(split_line),1):
                    temp.append(split_line[i])
                if len(temp)>0:
                    num_mid_material[int(split_line[2])] = []
                    num_mid_material[int(split_line[2])].append(temp)
            elif split_line[1] == 'Task':
                num_start_task[int(split_line[2])] = split_line[4]
                temp = []
                for i in range(5,len(split_line),1):
                    temp.append(split_line[i])
                if len(temp)>0:
                    num_mid_task[int(split_line[2])] = []
                    num_mid_task[int(split_line[2])].append(temp)
    ann_file.close()
                    
    f = open(output_file,'a')
    s = 0
    i = 0
    while i<len(text):
        increase = len(text[i])
        if s in num_start_process:
            key = clean_text(num_start_process[s])
            if key!= "":
                f.write(key+"\t"+"B-Process"+"\n")
            if s in num_mid_process:
                i = i + len(num_mid_process[s][0])
                for j in num_mid_process[s][0]:
                    key = clean_text(j)
                    if key!= "":
                        f.write(key+"\t"+"I-Process"+"\n")
                    s += len(j)+1

        elif s in num_start_material:
            key = clean_text(num_start_material[s])
            if key!= "":
                f.write(key+"\t"+"B-Material"+"\n")
            if s in num_mid_material:
                i = i + len(num_mid_material[s][0])
                for j in num_mid_material[s][0]:
                    key = clean_text(j)
                    if key!= "":
                        f.write(key+"\t"+"I-Material"+"\n")
                    s += len(j)+1


        elif s in num_start_task:
            key = clean_text(num_start_task[s])
            if key!= "":
                f.write(key+"\t"+"B-Task"+"\n")
            if s in num_mid_task:
                i = i + len(num_mid_task[s][0])
                for j in num_mid_task[s][0]:
                    key = clean_text(j)
                    if key!= "":
                        f.write(key+"\t"+"I-Task"+"\n")
                    s += len(j)+1

        else:
            key = clean_text(text[i])
            if key!= "":
                f.write(key+"\t"+"O"+"\n")
        s = s + increase+1
        i = i+1
    f.write("\n")
    f.close()

In [11]:
train_files = os.listdir("train/")
train_files.sort()
for i in range(0,len(train_files), 2):
    annotation_file = "train/"+train_files[i]
    txt_file = "train/"+train_files[i+1]
    output_file = "dataset/final_train.txt"
    try:
        preprocess(annotation_file, txt_file, output_file)
        
    except:
        print(annotation_file)


In [12]:
val_files = os.listdir("val/")
val_files.sort()
for i in range(0,len(val_files), 2):
    annotation_file = "val/"+val_files[i]
    txt_file = "val/"+val_files[i+1]
    output_file = "dataset/final_val.txt"
    try:
        preprocess(annotation_file, txt_file, output_file)
    except:
        print(annotation_file)


In [13]:
test_files = os.listdir("test/")
test_files.sort()
for i in range(0,len(test_files), 2):
    annotation_file = "test/"+test_files[i]
    txt_file = "test/"+test_files[i+1]
    output_file = "dataset/final_test.txt"
    try:
        preprocess(annotation_file, txt_file, output_file)
    except:
        print(annotation_file)
